In [59]:
import constants
from models import Character
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Pinecone
import pinecone

In [60]:
loader = TextLoader("./book2.txt")
book = loader.load()
print(f'Successfully loaded input of length {len(book)} with {len(book[0].page_content)} characters')

Successfully loaded input of length 1 with 437562 characters


In [61]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = splitter.split_documents(book)

In [67]:
embeddings = OpenAIEmbeddings(openai_api_key=constants.OPENAI_API_KEY)
pinecone.init(
    api_key=constants.PINECONE_API_KEY,
    environment=constants.PINECONE_ENVIRONMENT
)
index_name = "langchain"

In [70]:
search = Pinecone.from_texts([text.page_content for text in texts], embeddings, index_name=index_name)
# search = Pinecone.from_existing_index(index_name, embeddings)

In [71]:
llm = OpenAI(temperature=0.3, openai_api_key=constants.OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [73]:
# query = """
# List all of the characters in the book. Return an array of JSON objects in the following format: {"name": string, "bio": string}
# """
query = "List the full names of some characters from the book"
docs = search.similarity_search(query)
chain.run(input_documents=docs, question=query)


' Harry Potter, Ron Weasley, Hermione Granger, Ginny Weasley, Fred Weasley, George Weasley, Hagrid, Miranda Goshawk, Bathilda Bagshot, Adalbert Waffling, Emetic Switch, Phyllida Spore, Arsenius Jigger, Newt Scamander, Quentin Trimble, Lee Jordan, Mrs. Weasley.'

In [55]:
parser = PydanticOutputParser(pydantic_object=Character)

template = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = template.format_prompt(query="Who is the main character in the book?")

output = chain.run(input_documents=docs, question=prompt.to_string())

parser.parse(output)

OutputParserException: Failed to parse Character from completion  I don't know.. Got: Expecting value: line 1 column 1 (char 0)